# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://arlweb.msha.gov/drs/drshome.htm")

In [5]:
opname_input = driver.find_element_by_name('OperSearch')
driver.execute_script("arguments[0].scrollIntoView(true)", opname_input)

In [6]:
opname_input.send_keys("dirt")

In [7]:
button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
button.click()

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

In [21]:
operators = driver.find_elements_by_tag_name('tr')
for operator in operators[7:]:
    print('------------------')
    print(operator.text)


------------------
3503598
OR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC 
------------------
1401575
KS  Bender Sand & Dirt   BENDER SAND & DIRT Surface M  Intermittent  Construction Sand and Gravel 
------------------
5001797
AK  Dirt Company   Bush Pilot Surface M  Intermittent  Construction Sand and Gravel 
------------------
2103723
MN  Dirt Doctor Inc   Rock Lake Plant Surface M  Intermittent  Construction Sand and Gravel 
------------------
2103914
MN  Dirt Work Specialists LLC   Astec Plant Surface M  Intermittent  Construction Sand and Gravel 
------------------
4104757
TX  Dirt Works   Portable #1 Surface M  Intermittent  Construction Sand and Gravel 
------------------
0801306
FL  Holley Dirt Company, Inc   River Road Pit Surface M  Active  Sand, Common 
------------------
3901432
SD  Krueger Brothers Gravel & Dirt   PORTABLE SCREENER Surface M  Intermittent  Construction Sand and Gravel 
------------------
3609624
PA  M R Dirt   F

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

In [42]:
for operator in operators[7:-1]:
    tds = operator.find_elements_by_tag_name('td')
    print(tds[3].text)

Newberg Rock & Dirt
BENDER SAND & DIRT
Bush Pilot
Rock Lake Plant
Astec Plant
Portable #1
River Road Pit
PORTABLE SCREENER
Forbes Pit
Camptown Quarry
Fedscreek Surface
No. 3
Mine No. 6
Sandretto Drive
R D BLANKENSHIP DIRT WORK
Molino Pit
Pettibone Jaw Crusher
Chieftan 1400
Mike's Money Pit
Crusher


### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [44]:
for operator in operators[7:-1]:
    tds = operator.find_elements_by_tag_name('td')
    print(tds[0].text)

3503598
1401575
5001797
2103723
2103914
4104757
0801306
3901432
3609624
3609931
1519799
4407379
4407296
0203332
2901986
0801417
4300768
4300776
2302283
2103518


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [49]:
dirt_operators = []

for operator in operators[7:-1]:
    
    dirt_operator = {}
    
    tds = operator.find_elements_by_tag_name('td')
    
    ids = tds[0]
    dirt_operator['ID'] = ids.text
    
    states = tds[1]
    dirt_operator['State'] = states.text
    
    operator_names = tds[2]
    dirt_operator['Operator Name'] = operator_names.text
    
    mine_names = tds[3]
    dirt_operator['Mine Name'] = mine_names.text
    
    mine_types = tds[4]
    dirt_operator['Mine Type'] = mine_types.text
    
    cm = tds[5]
    dirt_operator['Coal or Metal'] = cm.text
    
    statuses = tds[6]
    dirt_operator['Status'] = statuses.text
    
    commodities = tds[7]
    dirt_operator['Commodity'] = commodities.text
     
    dirt_operators.append(dirt_operator)
    
dirt_operators


[{'ID': '3503598',
  'State': 'OR ',
  'Operator Name': 'Newberg Rock & Dirt  ',
  'Mine Name': 'Newberg Rock & Dirt',
  'Mine Type': 'Surface',
  'Coal or Metal': 'M ',
  'Status': 'Active ',
  'Commodity': 'Crushed, Broken Stone NEC '},
 {'ID': '1401575',
  'State': 'KS ',
  'Operator Name': 'Bender Sand & Dirt  ',
  'Mine Name': 'BENDER SAND & DIRT',
  'Mine Type': 'Surface',
  'Coal or Metal': 'M ',
  'Status': 'Intermittent ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '5001797',
  'State': 'AK ',
  'Operator Name': 'Dirt Company  ',
  'Mine Name': 'Bush Pilot',
  'Mine Type': 'Surface',
  'Coal or Metal': 'M ',
  'Status': 'Intermittent ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '2103723',
  'State': 'MN ',
  'Operator Name': 'Dirt Doctor Inc  ',
  'Mine Name': 'Rock Lake Plant',
  'Mine Type': 'Surface',
  'Coal or Metal': 'M ',
  'Status': 'Intermittent ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '2103914',
  'State': 'MN ',
  'Ope

### Save that to a CSV named `dirt-operators.csv`

In [55]:
import pandas as pd

df = pd.DataFrame(dirt_operators, columns=dirt_operator.keys())

In [58]:
df.to_csv("dirt-operators.csv", index=False)

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.

In [60]:
df = pd.read_csv("dirt-operators.csv")
df.head()

,ID,State,Operator Name,Mine Name,Mine Type,Coal or Metal,Status,Commodity
0,3503598,OR,Newberg Rock & Dirt,Newberg Rock & Dirt,Surface,M,Active,"Crushed, Broken Stone NEC"
1,1401575,KS,Bender Sand & Dirt,BENDER SAND & DIRT,Surface,M,Intermittent,Construction Sand and Gravel
2,5001797,AK,Dirt Company,Bush Pilot,Surface,M,Intermittent,Construction Sand and Gravel
3,2103723,MN,Dirt Doctor Inc,Rock Lake Plant,Surface,M,Intermittent,Construction Sand and Gravel
4,2103914,MN,Dirt Work Specialists LLC,Astec Plant,Surface,M,Intermittent,Construction Sand and Gravel
